## **Load files**

In [5]:
import pandas as pd

orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
category = pd.read_csv("../data/raw/product_category_name_translation.csv")

## **Convert Order Timestamps**

In [6]:
date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

> **Quick check** ~~Only If needed~~

In [7]:
orders[date_cols].info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_purchase_timestamp       99441 non-null  datetime64[us]
 1   order_delivered_customer_date  96476 non-null  datetime64[us]
 2   order_estimated_delivery_date  99441 non-null  datetime64[us]
dtypes: datetime64[us](3)
memory usage: 2.3 MB


## **Keep Only Delivered Orders (Business Logic)**
#### ***Why?***
- Cancelled orders distort revenue & delivery KPIs
- This is a real business assumption

In [9]:
orders["delivery_time_days"] = (
    orders["order_delivered_customer_date"] -
    orders["order_purchase_timestamp"]
).dt.days

orders["is_late_delivery"] = (
    orders["order_delivered_customer_date"] >
    orders["order_estimated_delivery_date"]
)

> **Quick check** ~~Only If needed~~

In [10]:
orders[["delivery_time_days", "is_late_delivery"]].describe()

,delivery_time_days
count,96476.000000
mean,12.094086
std,9.551746
min,0.000000
25%,6.000000
50%,10.000000
75%,15.000000
max,209.000000


## **Translate Product Categories to English**

In [11]:
products = products.merge(
    category,
    on="product_category_name",
    how="left"
)

products.rename(
    columns={"product_category_name_english": "category"},
    inplace=True
)


## **Minimal Cleaning**

In [12]:
orders.dropna(subset=["order_purchase_timestamp"], inplace=True)
items = items[items["price"] > 0]

## **Save Processed Files**

In [13]:
orders.to_csv("../data/processed/orders_clean.csv", index=False)
items.to_csv("../data/processed/order_items_clean.csv", index=False)
customers.to_csv("../data/processed/customers_clean.csv", index=False)
products.to_csv("../data/processed/products_clean.csv", index=False)
payments.to_csv("../data/processed/payments_clean.csv", index=False)
reviews.to_csv("../data/processed/reviews_clean.csv", index=False)

#### ***Done with***
- Clean data
- Business metrics
- Reusable processed tables